# Multi Feature-Rich Synthetic Colour (MFRSC) FOR POINT CLOUDS

This is a short tutorial to explain step by step the generation of synthetic colour for point clouds from various features according to the paper::

- Balado, J., González, E., Rodríguez-Somoza, J. L., & Arias, P. (2023). Multi feature-rich synthetic colour to improve human visual perception of point clouds. ISPRS Journal of Photogrammetry and Remote Sensing, 196, 514-527.

# Import of libraries

The libraries used in the mathematical morphology are *numpy*, *pyntcloud* and *pandas*.

- https://numpy.org/
- https://pyntcloud.readthedocs.io/en/latest/#
- https://pandas.pydata.org/


In [ ]:
import numpy as np
from pyntcloud import PyntCloud
import pandas as pd

# Reading data

Although there are many point cloud formats, in this tutorial we will start from *txt* clouds, which can be read using *numpy*, as they do not have any compression. The *txt* cloud is structured in 1 point per row and 1 attribute per column, and ' ' is specified as a delimiter between columns. 

In this case, the input cloud used for colouring contains in this order: 
- XYZ coordinates (3 columns)
- Intensity (1 column)
- Return number  (1 column)

As a working example a cloud scanned with Mobile Laser Scanner is provided.

In [ ]:
# Reading point cloud data
input_data = np.loadtxt("Nubes/pointcloud_XYZ_I_Rn.txt", delimiter=' ')

# Data extraction and conversion

Since colouring is based on features, it is necessary to extract them beforehand. The *pyntcloud* library offers easy and straightforward functions for feature extraction, but it is necessary to first convert the input data to a point cloud object.

For those unfamiliar with accessing point cloud data, both in *pandas* and other libraries, access is via [n rows, n columns], such that ":" indicates that all rows or columns are selected, and numerically "n:m" indicates that access is from row/column "n+1" to m.

In [ ]:
# Extract coordinate matrix from input data
coord = pd.DataFrame(list(zip(input_data[:,0],input_data[:,1],input_data[:,2])))  

# Assign column headings
coord.columns =['x', 'y', 'z']

# Visualisation
print(coord)

Convertir coordenadas a objeto nube de puntos

In [ ]:
# Conversion
cloud = PyntCloud(coord)

# Visualisation
print(cloud)

# Extraction of features from the input cloud

Some of the features using the MFRSC are available directly from the input data:

- ***Reflectance intensity*** (Re) is a radiometric feature provided by the LiDAR sensor. The intensity is a widely used to identify surfaces with high reflectivity such as road markings and traffic signs as well as to preserve textures due to variations in material and roughness.

- ***Return number*** (Rn) is a radiometric characteristic related to the penetration capacity of the laser in vegetation elements and crystals according to their wavelength. Return number feature is only available in multi-return LiDAR and the maximum number of returns is usually five for new LiDAR systems. This feature is widely used in the identification of vegetation cover.

- ***Depth*** (De) is the feature that provides a visualisation of horizontal distances. Depth is a very useful feature to identify objects by their silhouette (difference in distance between target and background).

- ***Height*** (He) is a feature provided by the point cloud at Z-coordinate. Height can be measured from sea level, in case of geo-referenced data. In either of these situations, in the subsequent normalization phase, the sea level offset is eliminated. Visualising height as a colour gradient allows the verticality and horizontality of environmental elements to be identified, a principle of human psychology to interpret scenes.

In [ ]:
# Reflectance
Intens = input_data[:,3]

# Return number
Return = input_data[:,4]

# Depth
Depth = np.sqrt(input_data[:,0]**2 + input_data[:,1]**2);

# Height
Height = input_data[:,2];

# Extraction of features from distances between points

For the following set of features it is necessary to calculate the neighbourhoods between points. The neighbourhood is calculated for *k* = 25 neighbours to obtain twin features in the following steps, although for the density only the 5 nearest neighbours are needed. 

In [ ]:
# Calculation of 25 neighbours
k_neighbors_25 = cloud.get_neighbors(k=25)

- ***Point density*** (Pd) is a feature that depends on the laser scanning frequency, the distance between the laser and the target surface and the angle of incidence. It is a relevant feature for highlighting or attenuating isolated points or areas with low point density. In this work we calculate point density based on the distance  between the first (d1) and fourth (d4) closest neighbouring points. This measure was proposed by Pfeifer et al. (2021). However, in order to obtain the delimited values (0–1), the division has been inverted.

In [ ]:
# Distance to first neighbour
d1 = np.sqrt(np.sum((input_data[:,0:3]-input_data[k_neighbors_25[:,0],0:3])**2,axis=1))

# Distance to the fith neighbour
d5 = np.sqrt(np.sum((input_data[:,0:3]-input_data[k_neighbors_25[:,4],0:3])**2,axis=1))

# Density calculation
Density = d1/d5

# Extraction of features from normals

Knowing the orientation of the surfaces that compose the point cloud is essential for estimating the slope:  

- ***Inclination*** (In) is the feature that indicates the orientation of the surface containing the point with respect to the horizon. The inclination is obtained from the calculation of the surface normal of the point regarding to *k* nearest neighbours. The first feature-based recognition model, known as the pandemonium proposed by Selfridge (1988), envisages that the visual system can have detectors of simple geometric features such as vertical, horizontal and oblique lines. Therefore, the visualisation of the inclination of surfaces in the point cloud can help in a more direct recognition.

In [ ]:
# Normal estimation
cloud.add_scalar_field("normals", k_neighbors=k_neighbors_25)

# Extraction of the normals from the point cloud object
normals = cloud.points[["nx(26)", "ny(26)", "nz(26)"]].to_numpy()

# Calculation of inclination
Inclination = np.absolute(np.arctan(np.sqrt(normals[:,0]**2+normals[:,1]**2)/normals[:,2]))*180/np.pi

# Extraction of features from eigenvalues

The remaining features are obtained from the eigenvalues of the point cloud:  

- ***Linearity*** (Li) is a geometric feature based on the distribution of *k* neighbouring points from the eigenvalues (Weinmann et al., 2015). Linearity enhances linear elements, e.g. pole like objects, but also corners between two planes. According to many object recognition theories, edges are one of the most important features. According to Marr and Nishihara’s theory, in the first stage of perception, the image is described as edges, spots, bars and the geometrical distribution. According to Biederman’s Recognition-by-Components (RBC) theory, geons are simple volumetric shapes and they responsible for object recognition. The first step of recognition would be to extract edges from changes in luminance and, in parallel, the division of the object into concave regions. The RBC/JIM model (Kurbat, 1994; Hummel and Biederman, 1992) consider that recognition occurs in a similar way to neural networks: activation of neurons in successive layers. In the first layer, edges are detected. In layers 2 and 3, geons, symmetry and blobs. In layers 4 and 5, size and orientation of geons. Given the importance of edges, linearity is a very relevant feature for object identification.

- ***Planarity*** (Pl) is a geometric feature calculated from the eigenvalues (Weinmann et al., 2015). Planarity enhances flat elements that conform most of the built environment and allow a visualisation of curvature. According to RBC theory, in addition to edge detection, in the construction of 3D representations also are relevant the non-accidental properties (symmetry, parallelism, straightness/curvature and connections), responsible for maintaining the constancy of objects. Therefore, planarity is relevant for identifying geons due to the visualisation of the curvature of the objects.

- ***Scattering*** (Sc) is a geometric feature calculated from the eigenvalues (Weinmann et al., 2015). Scattering enhances elements of irregular 3D shapes. This category includes vegetation as well as junctions of three or more planes, so that dispersion is presented as a feature also aligned with edge and geon identification.

In [ ]:
# Eigenvalue calculation
eigenvalues = cloud.add_scalar_field("eigen_values", k_neighbors=k_neighbors_25)

# Geometric feature estimation
cloud.add_scalar_field("linearity", ev=eigenvalues)
cloud.add_scalar_field("planarity", ev=eigenvalues)
cloud.add_scalar_field("sphericity", ev=eigenvalues)

# Visualisation of point cloud data
cloud.points

As can be seen from the justification above, there is no one-to-one correspondence between features and perceptual descriptors. Although point cloud features are carefully chosen, many features have interdependencies between them and cover several perceptual descriptors.Interdependences between point cloud features and perception descriptors are shown in the next table:

|   | Edges | Texture | Shape | Size | Depth | Orientation |
| ----------- | ----------- | ----------- | ----------- | ----------- | ----------- | ----------- |
| Reflectance |   | x | | | | x |
| Return number |   | x | | | |  |
| Depth | x |  | | x |x | x |
| Height | x |  | | x | x | x |
| Point density | x  | x | | | | |
| Inclination |   |  | x | | | x |
| Linearity |  x |  | x | | | |
| Planarity |  x |  | x | | | |
| Scattering |  x |  | x | | | |

# Normalisation of features

The selected features have different ranges, therefore a normalisation process is necessary to bound the features between 0 and 1. Some features such as density, linearity, planarity, and dispersion are already in this range. For the rest, a normalisation function is applied. For intensity and return number, saturation thresholds must be defined also. 

In [ ]:
# Normalise

Intens_n = Intens/1500

Return_n = Return/4

Depth_n = (Depth-np.min(Depth))/(np.max(Depth)-np.min(Depth))

Height_n = (Height-np.min(Height))/(np.max(Height)-np.min(Height))

Inclination_n = Inclination/90

# Generation of the feature matrix

For convenience,easy channel permutation, and possible later export, the calculated features are put together in a single matrix. 

In [ ]:
# Matrix of features
Features = np.column_stack((Intens_n,Return_n,Depth_n,Height_n,Density, cloud.points[['linearity(26)','planarity(26)','sphericity(26)']].to_numpy(),Inclination_n))


# Combination of characteristics per RGB channel

There are 362880 possible combinations of all features without repeating them. Evaluating the combination of all of them for each case study takes several hours of processing, so based on the paper presented by the MFRSC, all possible combinations have been tested and an optimal order of features has been suggested. 

The reduction is done through the conversion of 3 features to 1, according to the RGB to greyscale colour conversion.



<center> <img src="Figures/F02MFRSC.jpg"></center>
<center> Figure 1. Feature combination, nine to three RGB channels </center>

In [ ]:
# RGB
# Depth-Linear-Height
R = 0.2989 * Features[:,2] + 0.5870 * Features[:,5] + 0.1140 * Features[:,3];

# Return-Reflectance-Inclination 
G = 0.2989 * Features[:,1] + 0.5870 * Features[:,0] + 0.1140 * Features[:,8];

# Planar-Scatter-Density
B = 0.2989 * Features[:,6] + 0.5870 * Features[:,7] + 0.1140 * Features[:,4];

# Export point cloud with MFRSC

To save the cloud to disk, the *numpy* library and a *txt* cloud format are used. For saving, an address and name of the file to be generated are specified. The output point cloud contains the geometry (XYZ) of the input cloud and the calculated colours (RGB) in 0-1 scale. 

In [ ]:
# Definition of path and file name
ruta = "Nubes/pointcloud_MFRSC.txt"

# Data selection 
output_data = np.column_stack((input_data[:,0:3],R,G,B))

# Save
np.savetxt(ruta,output_data,delimiter=' ') 

This concludes the tutorial for generating MFRSC for point clouds. I actively recommend downloading the point cloud and viewing it on Cloud Compare. In addition, the last piece of code is the generation of a function for use in your own script without step-by-step explanation or intermediate outputs: 

In [ ]:
def MFRSC(XYZ,Intens,Return,Sat_Intens, Sat_Rn):
    # Extract coordinate matrix from input data
    coord = pd.DataFrame(list(zip(XYZ[:,0],XYZ[:,1],XYZ[:,2])))  

    # Assign column headings
    coord.columns =['x', 'y', 'z']
    
    # Conversion
    cloud = PyntCloud(coord)

    # Depth
    Depth = np.sqrt(XYZ[:,0]**2 + XYZ[:,1]**2);

    # Height
    Height = XYZ[:,2];
    
    # 25 neighbour calculation
    k_neighbors_25 = cloud.get_neighbors(k=25)

    # Distances
    d1 = np.sqrt(np.sum((XYZ[:,0:3]-XYZ[k_neighbors_25[:,0],0:3])**2,axis=1))    
    d5 = np.sqrt(np.sum((XYZ[:,0:3]-XYZ[k_neighbors_25[:,4],0:3])**2,axis=1))

    # Density
    Density = d1/d5

    # Surface normals
    cloud.add_scalar_field("normals", k_neighbors=k_neighbors_25)

    # Extraction of point cloud object normals
    normals = cloud.points[["nx(26)", "ny(26)", "nz(26)"]].to_numpy()

    # Inclination
    Inclination = np.absolute(np.arctan(np.sqrt(normals[:,0]**2+normals[:,1]**2)/normals[:,2]))*180/np.pi

    # Eigenvalues
    eigenvalues = cloud.add_scalar_field("eigen_values", k_neighbors=k_neighbors_25)

    # Geometric features
    cloud.add_scalar_field("linearity", ev=eigenvalues)
    cloud.add_scalar_field("planarity", ev=eigenvalues)
    cloud.add_scalar_field("sphericity", ev=eigenvalues)

    # Normalise
    Intens_n = Intens/Sat_Intens
    Return_n = Return/Sat_Rn
    Depth_n = (Depth-np.min(Depth))/(np.max(Depth)-np.min(Depth))
    Height_n = (Height-np.min(Height))/(np.max(Height)-np.min(Height))
    Inclination_n = Inclination/90

    Features = np.column_stack((Intens_n,Return_n,Depth_n,Height_n,Density, cloud.points[['linearity(26)','planarity(26)','sphericity(26)']].to_numpy(),Inclination_n))

    # Depth-Linear-Height
    R = 0.2989 * Features[:,2] + 0.5870 * Features[:,5] + 0.1140 * Features[:,3];

    # Return-Reflectance-Inclination 
    G = 0.2989 * Features[:,1] + 0.5870 * Features[:,0] + 0.1140 * Features[:,8];

    # Planar-Scatter-Density
    B = 0.2989 * Features[:,6] + 0.5870 * Features[:,7] + 0.1140 * Features[:,4];

    #Output data 
    output_data = np.column_stack((XYZ[:,0:3],R,G,B))

    return output_data

In [ ]:
# Example of application

# Select inputs
XYZ = input_data[:,0:3]
Intens = input_data[:,3]
Return = input_data[:,4]

# Call to MFRSC function
output_data = MFRSC(XYZ,Intens,Return,1500, 4)

# Save
ruta = "Nubes/pointcloud_MFRSC.txt"
np.savetxt(ruta,output_data,delimiter=' ') 